In [ ]:
import torchreid
model_name = 'resnet18'
dataset_name = 'soccernetv3'
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources=dataset_name,
    targets=dataset_name,
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset


Unzipping valid set to '/home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/reid-data/soccernetv3/reid' ...
Deleting valid set zip file at '/home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/reid-data/soccernetv3/reid/valid.zip'...
SoccerNet valid set is ready.


In [15]:
model = torchreid.models.build_model(
    name=model_name,
    num_classes=datamanager.num_train_pids, #2000 for Tango
    loss="softmax",
    pretrained=True
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [3]:
from tools.visualize_actmap import visactmap
test_loader = datamanager.test_loader
visactmap(model, test_loader, f"log/visactmap_{model_name}_{dataset_name}", 128, 256, True, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

Visualizing activation maps for market1501 ...


ValueError: The model output is supposed to have shape of (b, c, h, w), i.e. 4 dimensions, but got 3 dimensions. Please make sure you set the model output at eval mode to be the last convolutional feature maps

In [16]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [17]:
engine.run(
    save_dir=f"log/{model_name}_{dataset_name}",
    max_epoch=60,
    eval_freq=5,
    print_freq=100,
    test_only=False,
    visrank=False
)

=> Start training
epoch: [1/60][100/404]	time 0.012 (0.014)	data 0.000 (0.001)	eta 0:05:31	loss 6.0718 (6.4867)	acc 9.3750 (3.2188)	lr 0.000300
epoch: [1/60][200/404]	time 0.012 (0.013)	data 0.000 (0.001)	eta 0:05:16	loss 5.2781 (5.9913)	acc 6.2500 (6.8750)	lr 0.000300
epoch: [1/60][300/404]	time 0.012 (0.013)	data 0.000 (0.000)	eta 0:05:10	loss 4.3877 (5.5417)	acc 15.6250 (12.2188)	lr 0.000300
epoch: [1/60][400/404]	time 0.012 (0.013)	data 0.000 (0.000)	eta 0:05:06	loss 3.9319 (5.1082)	acc 40.6250 (19.0469)	lr 0.000300
epoch: [2/60][100/404]	time 0.013 (0.013)	data 0.000 (0.001)	eta 0:05:18	loss 3.2270 (3.0838)	acc 50.0000 (53.5938)	lr 0.000300
epoch: [2/60][200/404]	time 0.012 (0.013)	data 0.000 (0.001)	eta 0:05:07	loss 2.3503 (2.8779)	acc 75.0000 (58.8906)	lr 0.000300
epoch: [2/60][300/404]	time 0.013 (0.013)	data 0.000 (0.000)	eta 0:05:02	loss 1.9441 (2.7150)	acc 81.2500 (62.6562)	lr 0.000300
epoch: [2/60][400/404]	time 0.012 (0.013)	data 0.000 (0.000)	eta 0:04:59	loss 2.3661 (2.58

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 58.39it/s]


Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [00:02<00:00, 73.03it/s]


Done, obtained 15913-by-512 matrix
Speed: 0.0015 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 54.5%
CMC curve
Rank-1  : 76.8%
Rank-5  : 89.6%
Rank-10 : 93.4%
Rank-20 : 95.5%
Checkpoint saved to "log/resnet18_market1501/model/model.pth.tar-5"
epoch: [6/60][100/404]	time 0.012 (0.014)	data 0.000 (0.001)	eta 0:05:00	loss 1.2649 (1.3223)	acc 100.0000 (97.0312)	lr 0.000300
epoch: [6/60][200/404]	time 0.012 (0.013)	data 0.000 (0.001)	eta 0:04:47	loss 1.3567 (1.3488)	acc 90.6250 (96.3594)	lr 0.000300
epoch: [6/60][300/404]	time 0.012 (0.013)	data 0.000 (0.000)	eta 0:04:42	loss 1.2619 (1.3512)	acc 100.0000 (96.3854)	lr 0.000300
epoch: [6/60][400/404]	time 0.012 (0.013)	data 0.000 (0.000)	eta 0:04:39	loss 1.2059 (1.3535)	acc 100.0000 (96.2031)	lr 0.000300
epoch: [7/60][100/404]	time 0.012 (0.014)	data 0.000 (0.001)	eta 0:04:57	loss 1.3248 (1.3049)	acc 96.8750 (97.1250)	lr 0.000300
epoch: [7/60][200/404]	time 0.013 (0.013)	data 0.000 

...

Done, obtained 15913-by-512 matrix
Speed: 0.0015 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 65.8%
CMC curve
Rank-1  : 84.9%
Rank-5  : 93.7%
Rank-10 : 95.7%
Rank-20 : 97.2%
Checkpoint saved to "log/resnet18_market1501/model/model.pth.tar-55"
epoch: [56/60][100/404]	time 0.013 (0.014)	data 0.000 (0.001)	eta 0:00:26	loss 1.0345 (1.0360)	acc 100.0000 (100.0000)	lr 0.000003
epoch: [56/60][200/404]	time 0.013 (0.013)	data 0.000 (0.001)	eta 0:00:23	loss 1.0296 (1.0364)	acc 100.0000 (100.0000)	lr 0.000003
epoch: [56/60][300/404]	time 0.013 (0.013)	data 0.000 (0.000)	eta 0:00:22	loss 1.0613 (1.0368)	acc 100.0000 (100.0000)	lr 0.000003
epoch: [56/60][400/404]	time 0.012 (0.013)	data 0.000 (0.000)	eta 0:00:20	loss 1.0235 (1.0368)	acc 100.0000 (100.0000)	lr 0.000003
epoch: [57/60][100/404]	time 0.013 (0.013)	data 0.000 (0.001)	eta 0:00:20	loss 1.0254 (1.0383)	acc 100.0000 (100.0000)	lr 0.000003
epoch: [57/60][200/404]	time 0.013 (0.01

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 57.51it/s]


Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 160/160 [00:02<00:00, 72.89it/s]


Done, obtained 15913-by-512 matrix
Speed: 0.0014 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 65.8%
CMC curve
Rank-1  : 84.8%
Rank-5  : 93.7%
Rank-10 : 95.6%
Rank-20 : 97.1%
Checkpoint saved to "log/resnet18_market1501/model/model.pth.tar-60"
Elapsed 0:06:21
